## This program opens a folder with all of ur papers in pdf format and then analyzes them
 - isnt put in a spreasheet format yet.

This program below grabs all papers from the DB and prints them along with their contents and metadata

In [16]:
import boto3

# Initialize S3 client
s3 = boto3.client('s3')

# Specify your bucket name
bucket_name = 'generated-research'  # Replace with your bucket name

# List all objects in the bucket
response = s3.list_objects_v2(Bucket=bucket_name)

# Check if the bucket contains objects
if 'Contents' in response:
    for obj in response['Contents']:
        key = obj['Key']  # Object key (file name)

        # Fetch metadata for each object
        metadata_response = s3.head_object(Bucket=bucket_name, Key=key)
        metadata = metadata_response['Metadata']

        # Print file name and metadata
        print(f"File: {key}")
        print("Metadata:", metadata)

        # Download and print content of the object
        obj_response = s3.get_object(Bucket=bucket_name, Key=key)
        # content = obj_response['Body'].read().decode('utf-8')  # Assuming the content is text
        # print(f"Content:\n{content[:1000]}...")  # Print the first 1000 characters for testing
else:
    print("Bucket is empty.")


File: AND_ANALYSIS.txt
Metadata: {'author': 'human'}
File: Untitled_Paper.txt
Metadata: {'author': 'human'}
File: extracted-paper-test.txt
Metadata: {'author': 'human'}
File: extracted-paper-test2.txt
Metadata: {'author': 'human'}


This ongoing program is for grabbing new PDFs of papers and putting them in the database

In [15]:
import requests
import PyPDF2
from io import BytesIO
import re

# Initialize the S3 client
s3 = boto3.client('s3')

# Function to extract text from a PDF URL
def extract_text_from_pdf(url):
    try:
        # Send an HTTP request to the PDF URL
        response = requests.get(url)
        if response.status_code == 200:
            # Read the PDF content
            pdf_file = BytesIO(response.content)
            
            # Initialize PDF reader
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            
            # Extract text from all pages
            text = ''
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text += page.extract_text()

            return text
        else:
            print(f"Failed to retrieve PDF from {url}, Status Code: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error occurred while extracting PDF: {e}")
        return None

# Function to generate a title from the extracted PDF text
def generate_title_from_text(text):
    # Split the text into lines
    lines = text.split('\n')

    # Use the first non-empty line as a potential title
    for line in lines:
        line = line.strip()
        # Assuming the title is capitalized or the first meaningful line
        if line and len(line) > 10 and line.isupper():
            return line

    # Fallback if no title is found
    return "Untitled_Paper"

# Function to upload extracted text to S3 bucket with metadata
def upload_to_s3(bucket_name, file_name, text_content):
    try:
        # Convert text to bytes (S3 requires byte input)
        s3.put_object(
            Bucket=bucket_name, 
            Key=file_name, 
            Body=text_content, 
            Metadata={'author': 'human'}  # Adding metadata for the object
        )
        print(f"File '{file_name}' successfully uploaded to bucket '{bucket_name}' with metadata author='human'.")
    except Exception as e:
        print(f"Error occurred while uploading to S3: {e}")

# Example usage
pdf_url = 'https://doc.lagout.org/science/0_Computer%20Science/2_Algorithms/The%20Design%20and%20Analysis%20of%20Computer%20Algorithms%20%5BAho,%20Hopcroft%20&%20Ullman%201974-01-11%5D.pdf'
pdf_text = extract_text_from_pdf(pdf_url)

# Generate a title based on the content
if pdf_text:
    title = generate_title_from_text(pdf_text)
    file_name = f"{title}.txt".replace(" ", "_")  # Replace spaces with underscores for file name

    print(f"Generated Title: {title}")
    print(f"Extracted Text:\n{pdf_text[:1000]}...")  # Print the first 1000 characters of the text
    
    # Upload to S3 with metadata
    upload_to_s3('generated-research', file_name, pdf_text)
else:
    print("Failed to extract the PDF.")


Generated Title: AND ANALYSIS
Extracted Text:
THE DESIGN 
AND ANALYSIS 
OF 
COMPUTER 
ALGORITHMS 
Alfred V. Aho 
Bell Laboratories 
John E. Hopcroft 
Cornell University 
Jeffrey D. U II man 
Princeton University 
A " Addison-W~ley Publishing Company 
Reading. Massachusetts · Menlo Park. California 
London · Amsterdam · Don Mills. Ontario · Sydney PREFACE 
"' ...... 
The study of algorithms is at the very heart of c.:omputer science:- Irr; -redeht 
years a number of significant advances in the field of algorithms· have b'f'en 
made. These advances have ranged from the development off~s,te,r; algori,th~~. 
such as the fast Fourier transform. to the startling discovery of-q:rtain r)atur.~I. 
problems·for which all algorithms are inefficient. These results have kindled 
considerable interest in the study of algorithms. and the area of algorithm de­
sign and analysis has blossomed into a field of intense interest. The intent of 
this book is to bring together the fundamental results in this

In [1]:
import os
import re
import csv
import nltk
import PyPDF2
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from collections import Counter

# Download NLTK resources if you don't have them
# nltk.download('punkt')
# nltk.download('stopwords')

# Function to extract text from a PDF file
# def extract_text_from_pdf(pdf_path):
#     text = ""
#     with open(pdf_path, "rb") as file:
#         reader = PyPDF2.PdfReader(file)
#         for page_num in range(len(reader.pages)):
#             text += reader.pages[page_num].extract_text()
#     return text

folder_path = 'path/to/your/papers'
papers = []

for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as f:
            papers.append(f.read())

# Function to process text and compute the required metrics
def analyze_text(text, filename, csv_writer):
    print(f"Analyzing {filename}...")
    
    # Split into sentences using nltk
    sentences = nltk.sent_tokenize(text)
    
    # Tokenize words, and filter out punctuations
    words = nltk.word_tokenize(text)
    words = [word for word in words if word.isalpha()]
    
    # Get stopwords from NLTK
    stop_words = set(stopwords.words('english'))
    
    # Sentence length: average number of words per sentence
    sentence_lengths = [len(nltk.word_tokenize(sent)) for sent in sentences]
    avg_sentence_length = sum(sentence_lengths) / len(sentences) if sentences else 0

    # Word length: average number of characters per word
    word_lengths = [len(word) for word in words]
    avg_word_length = sum(word_lengths) / len(word_lengths) if word_lengths else 0

    # Comma frequency (for compound sentences)
    comma_count = text.count(',')

    # Paragraph length: average number of sentences per paragraph
    paragraphs = text.split('\n\n')  # Assuming paragraphs are separated by double newlines
    paragraph_lengths = [len(nltk.sent_tokenize(para)) for para in paragraphs]
    avg_paragraph_length = sum(paragraph_lengths) / len(paragraphs) if paragraphs else 0

    # Punctuation frequency (exclamation marks and question marks)
    punctuation_marks = re.findall(r'[!?]', text)
    punctuation_count = len(punctuation_marks)

    # Unique word count (proportional to length)
    unique_words = set(words)
    unique_word_proportion = len(unique_words) / len(words) if words else 0

    # Stop word frequency
    stop_word_count = sum(1 for word in words if word.lower() in stop_words)
    stop_word_frequency = stop_word_count / len(words) if words else 0

    # Top 10 words from Zipf's Law calculation
    top_10_words = calculate_zipfs_law(words, filename)
    
    # Writing the statistics to the CSV file
    csv_writer.writerow([
        filename,
        avg_sentence_length,
        avg_word_length,
        comma_count,
        avg_paragraph_length,
        punctuation_count,
        unique_word_proportion,
        stop_word_frequency,
        top_10_words
    ])

# Function to calculate Zipf's Law
def calculate_zipfs_law(words, filename):
    # Calculate word frequencies
    word_freq = Counter(words)
    
    # Sort words by frequency (highest to lowest)
    sorted_word_freq = word_freq.most_common()
    
    # Rank the words by frequency
    ranks = range(1, len(sorted_word_freq) + 1)
    frequencies = [freq for word, freq in sorted_word_freq]
    
    # Plotting Zipf's Law (log-log plot of rank vs frequency)
    plt.figure(figsize=(8, 6))
    plt.loglog(ranks, frequencies, marker="o", linestyle="none")
    plt.title(f"Zipf's Law: Word Frequency vs Rank ({filename})")
    plt.xlabel("Rank (log scale)")
    plt.ylabel("Frequency (log scale)")
    plt.grid(True)
    
    # Save the plot with the filename of the PDF
    plt.savefig(f"{filename}_zipf_plot.png")
    plt.close()

    # Return top 10 words as a string
    top_10_words = ', '.join([f"{word} ({freq})" for word, freq in sorted_word_freq[:10]])
    return top_10_words

# Function to process all PDFs in a folder and output the results to a CSV
def process_pdfs_in_folder(folder_path, csv_filename):
    # Open the CSV file for writing
    with open(csv_filename, mode='w', newline='', encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file)
        # Write the CSV header
        csv_writer.writerow([
            "Filename",
            "Avg Sentence Length (words)",
            "Avg Word Length (characters)",
            "Comma Frequency",
            "Avg Paragraph Length (sentences)",
            "Punctuation Frequency (!, ?)",
            "Unique Word Proportion",
            "Stop Word Frequency",
            "Top 10 Words (Zipf's Law)"
        ])
        
        # Iterate over each PDF in the folder
        for filename in os.listdir(folder_path):
            if filename.endswith(".pdf"):
                pdf_path = os.path.join(folder_path, filename)
                text = extract_text_from_pdf(pdf_path)
                analyze_text(text, filename, csv_writer)

# Example Usage
if __name__ == "__main__":
    folder_path = 'papers'  # Replace with the path to the folder containing the PDF files
    csv_filename = 'analysis_results.csv'  # The name of the output CSV file
    process_pdfs_in_folder(folder_path, csv_filename)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'path/to/your/papers'